In [2]:
# import numpy as np
# import math
# import cmath
# import pylab
# pylab.mpl.rcParams['font.sans-serif'] = ['SimHei']
# pylab.mpl.rcParams['axes.unicode_minus']=False    #用来正常显示负号
# ## 参数设置
# M = 256                 # 矩阵高度
# N = 256                 # 矩阵宽度
# top = M/8+1 
# bottom = M*7/8 
# left = N/8+1 
# right = N*7/8 
# theta = math.pi/12           # 扭曲或旋转角度

# ## 生成信号
# # 原始信号
# S0 = np.zeros((M,N))
# S0[int(top):int(bottom),int(left):int(right)] = 1 
# #S0[256/8+1:256*7/8 ,256/8+1:256*7/8] = 1 
# # 扭曲信号
# S1 = np.zeros((M,N)) 
# for ii in range(0,M):
#     for jj in range(0,N):
#         x = jj-N/2 
#         y = (M+1-ii)-M/2 
#         xx = round(x+N/2)
#         yy = M+1-round(x*math.sin(-theta)+y*math.cos(-theta)+M/2)
#         if(yy>=0 and yy<= M-1):
#             S1[ii,jj] = S0[yy,xx]
# # 旋转信号
# S2 = np.zeros((M,N)) 
# for ii in range(0,M):
#     for jj in range(0,N):
#         x = jj-N/2 
#         y = (M+1-ii)-M/2 
#         xx = round(x*math.cos(-theta)-y*math.sin(-theta)+N/2) 
#         yy = M+1-round(x*math.sin(-theta)+y*math.cos(-theta)+M/2) 
#         if(xx>=0 and xx<= N-1 and yy>=0 and yy<=M-1):
#             S2[ii,jj] = S0[yy,xx] 
# ## 二维傅里叶变换
# # 原始信号的二维傅里叶变换
# S0_ff = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(S0))) 
# S0_ff = abs(S0_ff) 
# S0_ff = S0_ff/S0_ff.max() 
# S0_ff = 20*np.log10(S0_ff+1e-4) 
# # 原始信号二维傅里叶变换
# S1_ff = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(S1))) 
# S1_ff = abs(S1_ff) 
# S1_ff = S1_ff/S1_ff.max() 
# S1_ff = 20*np.log10(S1_ff+1e-4) 
# # 原始信号二维傅里叶变换
# S2_ff = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(S2))) 
# S2_ff = abs(S2_ff) 
# S2_ff = S2_ff/S2_ff.max() 
# S2_ff = 20*np.log10(S2_ff+1e-4) 

# ## 画图
# pylab.figure(1)
# pylab.plt.subplot(2,3,1)
# pylab.pcolormesh(S0,cmap='jet')
# pylab.plt.gca().invert_yaxis()
# pylab.title('（a）时域，原始信号')
# pylab.plt.subplot(2,3,4)
# pylab.pcolormesh(S0_ff,cmap='jet')
# pylab.plt.gca().invert_yaxis()
# pylab.title('（b）原始信号频谱')
# pylab.plt.subplot(2,3,2)
# pylab.pcolormesh(S1,cmap='jet')
# pylab.plt.gca().invert_yaxis()
# pylab.title('（c）时域，扭曲信号')
# pylab.plt.subplot(2,3,5)
# pylab.pcolormesh(S1_ff,cmap='jet')
# pylab.plt.gca().invert_yaxis()
# pylab.title('（d）扭曲信号频谱')
# pylab.plt.subplot(2,3,3)
# pylab.pcolormesh(S2,cmap='jet')
# pylab.plt.gca().invert_yaxis()
# pylab.title('（e）时域，旋转信号')
# pylab.plt.subplot(2,3,6)
# pylab.pcolormesh(S2_ff,cmap='jet')
# pylab.plt.gca().invert_yaxis()
# pylab.title('（f）旋转信号频谱')
# pylab.show()

In [3]:
import scipy as sci
import numpy as np
import math

In [6]:
c = 3e8   #光速
fc = 5e9   #载波频率
B = 200e6;  #带宽
lamda = c/fc   #波长
Tp = 1.5e-6    #脉宽
Kr = B/Tp   #调频率
fs = 1.6*B    #采样率

H = 200    #飞机高度
Ls = 200    #合成孔径长度
v = 100    #飞机速度
Lt = Ls/v    #合成孔径时间

# 成像区域[Xc-X0,Xc+X0; Yc-Y0,Yc+Y0]
# 以合成孔径中心为原点，距离向为x轴，方位向为y轴
Xc = 10000
Yc = 0
Xo = 100
Yo =100

Rc =np.sqrt(H^2+Xc^2)     #中心距离
Ka = 2*v**2/(Rc*lamda)   #多普勒调频率
Bmax = Lt*Ka           #多普勒最大带宽
fa = math.ceil(3*Bmax)      #脉冲重复频率

In [57]:
Rmin = np.sqrt(H^2+(Xc-Xo)^2)   #观测场景距飞机的最近距离
Rmax = np.sqrt((Xc+Xo)**2+H**2+(Yc+Yo+Ls/2)**2)  #最远距离
rm = Ls + 2*Yo    #雷达走过的总路程长度
tm= np.arange(0,rm/v-1/fa,1/fa)               #慢时间（合成孔径时间+成像区域时间）
tk= np.arange(2*Rmin/c-Tp/2,2*Rmax/c-1/fs+Tp/2,1/fs)       #快时间（距离门内）
tk = np.array([tk])
# print(len(tm))
# print(len(tk))
# print(len(target))
target = np.array([[Xc,Yc,0],[Xc+80,Yc+45,0],[Xc-20,Yc-20,0]])        #目标坐标
echo = np.zeros((len(tm),len(tk),len(target)))  #回波
echo_all = np.zeros((len(tm),len(tk)));
y = -v*(rm/v)/2+v*tm              #飞机y轴坐标
print(y.shape)
print(tk.shape)
print(tk.dtype)
print(tk)
print(tm.shape)

(79211,)
(1, 21823)
float64
[[-8.90915343e-08 -8.59665343e-08 -8.28415343e-08 ...  6.80984085e-05
   6.81015335e-05  6.81046585e-05]]
(79211,)


In [43]:
def rectpulse(x,Nsamp):
    '''
    RECTPULSE Rectangular pulse shaping.     矩形脉冲矩形脉冲整形。
    Y = RECTPULSE(X,NSAMP) returns Y, a rectangular pulse shaped version of X, X的矩形脉冲形版本
    with NSAMP samples per symbol. This function replicates each symbol in  每个符号具有NSAMP样本。此函数以X NSAMP次复制每个符号。
    X NSAMP times. To insert zeros between each sample of X, see UPSAMPLE. 要在X的每个采样之间插入零，请参见上采样。
    For two-dimensional signals, the function treats each column as 1 channel. 对于二维信号，该函数将每列视为1个通道。


    See also INTDUMP, UPSAMPLE, GAUSSFIR.      另请参见INTDUMP、UPSAMPLE、GAUSSFIR。

    Copyright 1996-2011 The MathWorks, Inc.

    Check x, Nsamp
    '''
    (wid, leng) = x.shape
    if ((wid == 1)&(len!=1)):
        a1 = np.ones((Nsamp, 1)) * x.reshape((1, wid * leng))
        y = (np.ones((Nsamp, 1)) * x.reshape((1, wid * leng))).reshape((wid, leng * Nsamp))
    else:
        a1 = np.ones((Nsamp, 1)) * x.reshape((1, wid * leng))
        y = (np.ones((Nsamp, 1)) * x.reshape((1, wid * leng))).reshape((wid * Nsamp,leng))

    return y
# def rect_wave(x,c,c0):     #起点为c0，宽度为c的矩形波
#     if x>=(c+c0):
#           r=0.0
#     elif x<c0:
#           r=0.0
#     else:
#           r=1
#     return r

In [58]:
for k in range(1,4):  #目标数
    for i in range(1,len(tm)+1):   #慢时间轴
        radar = np.array([0,y[i-1],H])      #飞机坐标
        Rtm = np.sqrt(sum((target[k-1]-radar)**2))
        arr = np.abs(target[k-1,2]-y[i-1])/Xc
        tk-2*Rtm/c
        echo[i,:,k-1]=arr[arr<0.01]*rectpulse(tk-2*Rtm/c,int(Tp*fs))*np.exp(1j*2*np.pi*fc*(tk-2*Rtm/c)+1j*np.pi*Kr*(tk-2*Rtm/c)**2) #回波模型
              #注意上边为什么是tk-tr(i)-Tp/2，而不是tk-tr(i)
    echo_all = echo[:,:,k] + echo_all

ValueError: operands could not be broadcast together with shapes (0,) (1,10475040) 

In [51]:
for ii in range(1,5):
    print(ii)
    

1
2
3
4
